## mongoDb  is full:
```
 * more than 900k urls to_crawl
 * >160K crawled_urls
 * (500MB) Mongo free
```
#### Solution:
* only keep 100k-200K to_crawls in mongo at a time
* remove crawled_urls
* Avoid crawling data urls

get all to_crawl (920054)   | Save to csv
get all crawled  (160842)   | Save to csv



* make scrapy_engine_spider to push to "to_crawl?" instead of "to_crawl"
* avoid updating "crawled" from scrapy_spider to mongo

* Get Urls from "to_crawl?"
    [ ] Store Crawled urls in bloom function.
    [ ] bloom filter to check urls in "to_crawl?" does not exist in urls_crawled
    [X] store to csv/sqlite (avoid duplicate): csv files: "to_crawl", "crawled" or sqlite_tables
    [X] Delete to_crawls and crawled from mongo

    if len(to_crawl_in_mongo < 100000):
        [ ] shuffle(to_crawl_urls)
        [X] add 100k to_crawl urls from local to mongo
        [X] delete all attempted mongo inserts.
        [ ] delete only successful mongo insert and store unsuccessful ones somewhere

update to_crawl and "crawled" using crawled_data
[X] for data in crawled_data:
        if data['parent_url'] in to_crawl:
            remove data['parent_url'] from to_crawl
        append_to_crawled(data['parent_url'])




Note:
* "to_crawl?" -> uploaded by scrapy spider to mongo
* "to_crawl"  -> actual to_crawl link uploaded to mongo by server
* sqlite seems to handle concurrency by itself without throwing errors.

In [4]:
from mongo import Mongo
mongo = Mongo()

# Get 10 data urls from to_crawl.
urls = mongo.collection.find({"status": 'to_crawl?'}).limit(10)

# get urls starting with 'data:'
data_urls = mongo.collection.find({"url": {"$regex": "^data:"}})
list(data_urls)

[]

In [11]:
mongo.collection.count_documents({"status": 'to_crawl'})

920054

In [3]:
from mongo import Mongo
db = Mongo()
# Count Entries with status="to_crawl"
db.collection.find({"status": "to_crawl"}).count()

AttributeError: 'Cursor' object has no attribute 'count'

##### Get all urls from mongo and save to sqlite

In [ ]:
from mongo import Mongo
mongo = Mongo()
# get all to_crawl urls 50000 at a time
# Initialize skip to 0
skip = 0
bulk_size = 50000

the_to_crawl_urls = []
while True:
    # Get the next 50000 documents
    to_crawl_urls = mongo.collection.find({"status": 'to_crawl'}).skip(skip).limit(bulk_size)
    # Convert the cursor to a list
    crawled_urls_list = list(to_crawl_urls)
    # If the list is empty, break the loop
    if not crawled_urls_list:
        break
    # Process the documents
    the_to_crawl_urls.extend([(url['url'], url['timestamp']) for url in crawled_urls_list])
    # Increase skip by 50000
    skip += bulk_size
    print(skip)

len(the_to_crawl_urls)

In [8]:
# Save to sqlite
from sqlite_handler import URLDatabase
url_db = URLDatabase(db_path="urls.db")

# Insert the data into the database
url_db.bulk_insert("to_crawl", the_to_crawl_urls)

url_db.count("to_crawl")

# # Get all the urls from the database
# urls = url_db.fetch('to_crawl', 10)
# urls

In [ ]:
from mongo import Mongo
mongo = Mongo()
# get all crawled urls 50000 at a time
# Initialize skip to 0
skip = 0
bulk_size = 50000

the_crawled_urls = []
while True:
    # Get the next 50000 documents
    crawled_urls = mongo.collection.find({"status": 'crawled'}).skip(skip).limit(bulk_size)
    # Convert the cursor to a list
    crawled_urls_list = list(crawled_urls)
    # If the list is empty, break the loop
    if not crawled_urls_list:
        break
    # Process the documents
    the_crawled_urls.extend([(url['url'], url['timestamp']) for url in crawled_urls_list])
    # Increase skip by 50000
    skip += bulk_size
    print(skip)

len(the_crawled_urls)

In [ ]:
# Save to sqlite
from sqlite_handler import URLDatabase
url_db = URLDatabase(db_path="urls.db")

# Insert the data into the database
url_db.bulk_insert("crawled", the_crawled_urls)


import time;start=time.time();a=url_db.fetch('crawled', 100000);print(start-time.time())

# # Get all the urls from the database
urls = url_db.fetch('crawled', 10)

In [15]:
from mongo import Mongo
mongo = Mongo()


sample_url = {'url': 'https://wdww.iana.org/f_img/2013.1/iana-logo-header.svg1', 'status': 'to_crawl?', 'timestamp': '2021-07-01T00:00:00.000Z'}
mongo.collection.insert_one(sample_url)

# Get all urls from "to_crawl?"
urls = mongo.collection.find({"status": 'to_crawl?'})
to_crawl_urls = [(url['url'], url['timestamp']) for url in list(urls)]
to_crawl_urls

# Remove multiple url from "to_crawl?"
mongo.collection.delete_many({"status": 'to_crawl?', "url": {"$in": [url[0] for url in to_crawl_urls]}})


DeleteResult({'n': 3, 'electionId': ObjectId('7fffffff0000000000000405'), 'opTime': {'ts': Timestamp(1716833546, 33), 't': 1029}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1716833546, 35), 'signature': {'hash': b"\xa7.\x84\xd8K\x97\xd6\xbc\xf5\x08W'\x06\xe1o\x9b\xa0\xaf\x1cl", 'keyId': 7318892626235621378}}, 'operationTime': Timestamp(1716833546, 33)}, acknowledged=True)

In [1]:
# Get all urls from "to_crawl?"
urls = mongo.collection.find({"status": 'to_crawl?'})
to_crawl_urls = [(url['url'], url['timestamp']) for url in list(urls)]
to_crawl_urls

# Save to sqlite
from sqlite_handler import URLDatabase
url_db = URLDatabase(db_path="urls.db")

# url_db.exists("crawled", to_crawl_urls[0][0])

# Insert the data into the database
url_db.bulk_insert("crawled", to_crawl_urls)

# Delete 
# url_db.count_entries("crawled")

# url_db.fetch_all("crawled")

# # Get all the urls from the database
# # urls = url_db.fetch_all('test')

NameError: name 'to_crawl_urls' is not defined

##### Final Code


In [13]:
'''
* creating a giant thread to avoid concurrency issues
'''
import sys
from mongo import Mongo
mongo = Mongo()
from sqlite_handler import URLDatabase
url_db = URLDatabase(db_path="urls.db")


# Get all urls from "to_crawl?"
urls = mongo.collection.find({"status": 'to_crawl?'})
to_crawl_urls = [(url['url'], url['timestamp']) for url in list(urls)]
# to_crawl_urls

# Save to sqlite
# Insert the data into the database
url_db.bulk_insert("crawled", to_crawl_urls)

# delete from mongo
mongo.collection.delete_many([{'url': url[0], 'status': 'to_crawl?'} for url in to_crawl_urls])


if mongo.collection.count_documents({"status": 'to_crawl'}) < 100000:
    new_to_crawl_urls = url_db.fetch('to_crawl', 100000)
    n_failed_to_upload = 0
    try:
        # insert many
        mongo.collection.insert_many([{'url': url[0], 'status': 'to_crawl', 'timestamp': url[1]} for url in new_to_crawl_urls], ordered=False)
    except Exception as bwe:
        # pass
        # Get the details of the operations that failed
        failed_ops = bwe.details['writeErrors']
        

        # Get the documents that failed to insert
        failed_docs = [op['op'] for op in failed_ops]

        # Get the URLs that failed to insert
        urls_failed_to_upload_to_mongo = [(doc['url'], doc['timestamp']) for doc in failed_docs]
        n_failed_to_upload = len(urls_failed_to_upload_to_mongo)
        # for url in urls_failed_to_upload_to_mongo:
        #     logging.error(f"Failed to upload {url} to MongoDB")
        # success_urls = [url for url in new_to_crawl_urls if url not in urls_failed_to_upload_to_mongo]

        # # Delete successful urls from sqlite
        # url_db.delete("to_crawl", success_urls)
        # # print(f'success_urls:{success_urls}, len:{len(success_urls)}')
        # print(failed_urls, len(failed_urls))
    # # delete from sqlite
    if n_failed_to_upload < 10000:
        url_db.delete("to_crawl", new_to_crawl_urls)
        # print(n_failed_to_upload)
    else:
        print(f'failed to upload {n_failed_to_upload} urls to mongo')
        # exit the python script
        sys.exit(1)
        


1463


In [1]:
from mongo import Mongo
mongo = Mongo()

# Delete all urls with status 'crawled'
# mongo.collection.delete_many({"status": 'to_crawl'})
mongo.collection.delete_many({"status": 'crawled'})
mongo.collection.delete_many({"status": 'test'})
mongo.collection.delete_many({"status": 'to_crawl?'})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000405'), 'opTime': {'ts': Timestamp(1716835666, 11), 't': 1029}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1716835666, 11), 'signature': {'hash': b'a\x0cbO\xfd"\xda\xff0,*\xbaK&\xad@\xba\xb5\xe6o', 'keyId': 7318892626235621378}}, 'operationTime': Timestamp(1716835666, 11)}, acknowledged=True)

### DB stats

In [5]:
from mongo import Mongo
mongo = Mongo()

# crawled_count = mongo.collection.count_documents({"status": 'crawled'})
to_crawl_spider_count = mongo.collection.count_documents({"status": 'to_crawl?'})
to_crawl_spider_count

ConfigurationError: The resolution lifetime expired after 21.125 seconds: Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.

In [3]:
crawled_count

DeleteResult({'n': 4565, 'electionId': ObjectId('7fffffff0000000000000405'), 'opTime': {'ts': Timestamp(1716866814, 4581), 't': 1029}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1716866814, 4585), 'signature': {'hash': b'\x8c\x88S3\x07\xd2\xa6E\x94\xe6\xbc@\xfe\xa8\xb4$\x06\xc9\x05>', 'keyId': 7318892626235621378}}, 'operationTime': Timestamp(1716866814, 4581)}, acknowledged=True)